In [1]:
import pandas as pd
import geopandas as gpd
import rioxarray
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import math
import multiprocessing as mp

In [2]:
landscan = gpd.read_file('/Users/kpierce/CooksProTX/texas_avg_pop_30arcsec_with_tract_id/texas_avg_pop_30arcsec_with_tract_id.shp')

Assign an index ID to each value from the landscan raster:

In [55]:
landscan['pixel_id'] = [i for i in range(landscan.shape[0])]

In [56]:
landscan.head()

,x,y,avg_popula,index_righ,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,pixel_id
0,-97.391667,25.838333,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.39167 25.83833),0
1,-97.425000,25.846667,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.42500 25.84667),1
2,-97.416667,25.846667,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.41667 25.84667),2
3,-97.408333,25.846667,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.40833 25.84667),3
4,-97.400000,25.846667,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.40000 25.84667),4


In [16]:
svi = pd.read_csv('/Users/kpierce/protxdb/data/raw/final_demographic_data/all_demographic_features.csv', dtype={'GEOID': str})
op = pd.read_csv('/Users/kpierce/protxdb/data/raw/final_demographic_data/opportunity_data.csv', dtype={'GEOID': str})
com = pd.read_csv('/Users/kpierce/protxdb/data/raw/final_demographic_data/binned_commute_times.csv', dtype={'GEOID': str})

In [23]:
svi_tract = svi[(svi['GEO_TYPE'] == 'tract') & (svi['YEAR'] == 2019)]
op_tract = op[(op['GEO_TYPE'] == 'tract') & (op['YEAR'] == 2019)]
com_tract = com[(com['GEO_TYPE'] == 'tract') & (com['YEAR'] == 2019)] 

In [24]:
svi_pct = [i for i in svi_tract.columns if 'EP' in i]
svi_tract_pct = svi_tract[['GEOID'] + svi_pct]

op_pct = [i for i in op_tract.columns if 'EP' in i]
op_tract_pct = op_tract[['GEOID'] + op_pct]

com_pct = [i for i in com_tract.columns if 'EP' in i]
com_tract_pct = com_tract[['GEOID'] + com_pct]

In [25]:
svi_op = pd.merge(svi_tract_pct, op_tract_pct, on='GEOID')
all_vars = pd.merge(svi_op, com_tract_pct, on='GEOID')

In [26]:
all_vars.columns

Index(['GEOID', 'EP_DISABL', 'EP_MOBILE', 'EP_NOHSDP', 'EP_NOVEH', 'EP_POV',
       'EP_UNEMP', 'EP_CROWD', 'EP_LIMENG', 'EP_AGE17', 'EP_AGE65',
       'EP_GROUPQ', 'EP_MINRTY', 'EP_MUNIT', 'EP_SNGPNT', 'EP_UNINSUR',
       'EP_AMERICAN_INDIAN_ALASKA_NATIVE_ALONE', 'EP_ASIAN_ALONE',
       'EP_BLACK_AFRICAN_AMERICAN_ALONE', 'EP_HISPANIC_LATINO',
       'EP_NATIVE_HAWAIIAN_OTHER_PACIFIC_ISLANDER_ALONE',
       'EP_OTHER_RACE_ALONE', 'EP_TWO_OR_MORE_RACES',
       'EP_WHITE_ALONE_NOT_HISPANIC_LATINO', 'EP_WHITE_ALONE',
       'EP_FOREIGN_BORN', 'EP_RENTER_OCCUPIED_HU', 'EP_10_14_MIN',
       'EP_15_19_MIN', 'EP_20_24_MIN', 'EP_25_29_MIN', 'EP_30_34_MIN',
       'EP_35_39_MIN', 'EP_40_44_MIN', 'EP_45_59_MIN', 'EP_5_9_MIN',
       'EP_5LESS_MIN', 'EP_60_89_MIN', 'EP_90PLUS_MIN', 'EP_COMMUTE_POP',
       'EP_30LESS_MIN', 'EP_30_60_MIN', 'EP_60PLUS_MIN'],
      dtype='object')

In [27]:
landscan.shape

(951054, 17)

In [28]:
all_vars.shape

(5265, 43)

In [34]:
all_vars_long = all_vars.melt(id_vars='GEOID', var_name='DEMOGRAPHICS_NAME', value_name='VALUE')

In [35]:
all_vars_long.shape

(221130, 3)

In [57]:
landscan_vars = gpd.GeoDataFrame(
    pd.merge(landscan, all_vars_long, on='GEOID', how='left'),
    crs=landscan.crs
)

In [58]:
landscan_vars.shape

(39944268, 20)

In [59]:
landscan_vars.head()

,x,y,avg_popula,index_righ,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,pixel_id,DEMOGRAPHICS_NAME,VALUE
0,-97.391667,25.838333,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.39167 25.83833),0,EP_DISABL,7.3
1,-97.391667,25.838333,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.39167 25.83833),0,EP_MOBILE,20.3
2,-97.391667,25.838333,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.39167 25.83833),0,EP_NOHSDP,46.5
3,-97.391667,25.838333,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.39167 25.83833),0,EP_NOVEH,2.2
4,-97.391667,25.838333,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,G5020,S,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.39167 25.83833),0,EP_POV,28.6


In [60]:
landscan_vars.columns

Index(['x', 'y', 'avg_popula', 'index_righ', 'STATEFP', 'COUNTYFP', 'TRACTCE',
       'GEOID', 'NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER',
       'INTPTLAT', 'INTPTLON', 'geometry', 'pixel_id', 'DEMOGRAPHICS_NAME',
       'VALUE'],
      dtype='object')

In [61]:
landscan_vars['value_fraction'] = [i/100 for i in landscan_vars['VALUE']]

In [62]:
landscan_vars['ADJUSTED_VALUE'] = landscan_vars['value_fraction'] * landscan_vars['avg_popula']

In [63]:
landscan_vars.head()

,x,y,avg_popula,index_righ,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,...,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,pixel_id,DEMOGRAPHICS_NAME,VALUE,value_fraction,ADJUSTED_VALUE
0,-97.391667,25.838333,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,...,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.39167 25.83833),0,EP_DISABL,7.3,0.073,0.0
1,-97.391667,25.838333,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,...,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.39167 25.83833),0,EP_MOBILE,20.3,0.203,0.0
2,-97.391667,25.838333,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,...,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.39167 25.83833),0,EP_NOHSDP,46.5,0.465,0.0
3,-97.391667,25.838333,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,...,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.39167 25.83833),0,EP_NOVEH,2.2,0.022,0.0
4,-97.391667,25.838333,0.0,2239,48,061,014100,48061014100,141,Census Tract 141,...,58880502.0,1656391.0,+25.8820153,-097.4007135,POINT (-97.39167 25.83833),0,EP_POV,28.6,0.286,0.0


In [64]:
landscan_keep = landscan_vars[['pixel_id', 'GEOID', 'DEMOGRAPHICS_NAME', 'ADJUSTED_VALUE']]

In [65]:
landscan_keep.head()

,pixel_id,GEOID,DEMOGRAPHICS_NAME,ADJUSTED_VALUE
0,0,48061014100,EP_DISABL,0.0
1,0,48061014100,EP_MOBILE,0.0
2,0,48061014100,EP_NOHSDP,0.0
3,0,48061014100,EP_NOVEH,0.0
4,0,48061014100,EP_POV,0.0


In [66]:
landscan_vars_wide = landscan_keep.pivot(index='pixel_id', columns='DEMOGRAPHICS_NAME', values='ADJUSTED_VALUE')

In [68]:
landscan_vars_wide = landscan_vars_wide.reset_index()

In [69]:
landscan_vars_wide.head()

DEMOGRAPHICS_NAME,pixel_id,EP_10_14_MIN,EP_15_19_MIN,EP_20_24_MIN,EP_25_29_MIN,EP_30LESS_MIN,EP_30_34_MIN,EP_30_60_MIN,EP_35_39_MIN,EP_40_44_MIN,...,EP_NOVEH,EP_OTHER_RACE_ALONE,EP_POV,EP_RENTER_OCCUPIED_HU,EP_SNGPNT,EP_TWO_OR_MORE_RACES,EP_UNEMP,EP_UNINSUR,EP_WHITE_ALONE,EP_WHITE_ALONE_NOT_HISPANIC_LATINO
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
landscan_vars_final = gpd.GeoDataFrame(
    pd.merge(landscan, landscan_vars_wide, on='pixel_id', how='left'),
    crs=landscan.crs
)

In [72]:
landscan_vars_final.tail()

,x,y,avg_popula,index_righ,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,...,EP_NOVEH,EP_OTHER_RACE_ALONE,EP_POV,EP_RENTER_OCCUPIED_HU,EP_SNGPNT,EP_TWO_OR_MORE_RACES,EP_UNEMP,EP_UNINSUR,EP_WHITE_ALONE,EP_WHITE_ALONE_NOT_HISPANIC_LATINO
951049,-100.808333,36.380000,800.0,4703,48,357,950400,48357950400,9504,Census Tract 9504,...,12.000,28.823184,106.400,227.793534,62.166761,38.065400,48.800,191.200,722.146074,362.482867
951050,-100.825000,36.388333,104.0,4703,48,357,950400,48357950400,9504,Census Tract 9504,...,1.560,3.747014,13.832,29.613159,8.081679,4.948502,6.344,24.856,93.878990,47.122773
951051,-100.816667,36.388333,1022.0,4703,48,357,950400,48357950400,9504,Census Tract 9504,...,15.330,36.821617,135.926,291.006239,79.418037,48.628549,62.342,244.258,922.541610,463.071862
951052,-100.808333,36.388333,757.0,4703,48,357,950400,48357950400,9504,Census Tract 9504,...,11.355,27.273938,100.681,215.549631,58.825298,36.019385,46.177,180.923,683.330723,342.999413
951053,-100.808333,36.396667,933.0,4703,48,357,950400,48357950400,9504,Census Tract 9504,...,13.995,33.615038,124.089,265.664209,72.501985,44.393773,56.913,222.987,842.202859,422.745643


In [73]:
landscan_vars_final.to_file('/Users/kpierce/CooksProTX/demographic_data_landscan_30arcsec.shp')